# Preprocessing Data

- The `sklearn` library contains a lot of efficient tools for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction
- Keras is a high-level, deep learning API developed by Google for implementing neural networks
- Pandas is a Python library used for working with data sets. It has functions for analyzing, cleaning, exploring, and manipulating data.
- XGBoost is a popular and efficient open-source implementation of the gradient boosted trees algorithm
- `pyvi`: Python Vietnamese Toolkit.
- `tqdm`: used to create a smart progress bar for the loops
- `numpy`: a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays

# Dataset Preparation

In [1]:
from pyvi import ViTokenizer, ViPosTagger
from tqdm import tqdm
import numpy as np
import gensim
import pickle

## Diference between os.getcwd() and os.path.dirname(__file__)
There is a difference, though you wouldn't be able to tell from a single script.

`__file__` is the full filename of a loaded module or script, so getting the parent directory of it with `os.path.dirname(__file__)` gets you the directory that script is in.

Note: on Linux (and similar OSes), such a filename can be a symbolic link to the actual file which may reside somewhere else. You can use `os.path.realpath()` to resolve through any such links, if needed, although you can typically use the symlink equivalently. On Windows these are less common, but similarly, you can resolve symbolic links through `realpath()`.

`os.getcwd()` gets you the current working directory. If you start a script from the directory the script is in (which is common), the working directory will be the same as the result from the call from `os.path.dirname(__file__)`.

But if you start the script from another directory (i.e. python `d:\some\path\script.py`), or if you change the working directory during the script (e.g. with `os.chdir()`), the current working directory has changed, but the directory part of the script filename has not.

So, it depends on what you need:
- Need the directory your script file is in? Use `os.path.dirname(__file__)`
- Need the directory your script is currently running in? use `os.getcwd()`

You'll see / in some results and \ in others. Sadly, MS Windows uses \ to separate parts of a path (e.g. `C:\Program Files\App\`), while pretty much all other operating systems use / (e.g. `/home/user/script.py`)

Python will often convert those automatically, so you can use paths like `C:/Program Files/App` in Python on Windows as well, but it tends to be a good idea to be safe and use `os.path.sep`.

Note: if you're on Python 3, you may be better off just using pathlib's Path instead of os.path. It automatically resolves symbolic links (although you can still resolve to the link if you prefer) and has other nice conveniences as well.



In [2]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'project 2')
# '/Users/macos/Desktop/Github/NLP/Text Classifier'
# Load data from dataset folder
# VNTC-master/Data/10Topics/Ver1.1/Train_Full
# VNTC-master/Data/10Topics/Ver1.1/Test_Full
def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in dirs:
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)
#                 sentence = ' '.join(words)
#                 print(lines)
                X.append(lines)
                y.append(path)
#             break
#         break
    return X, y

> Chỉ chạy block code này một lần đầu để lưu đọc dữ liệu. Sau đó lưu vào file .pkl thì các lần sau chỉ việc lấy ra từ file đó

In [9]:
# train_path = os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Train_Full')
# X_data, y_data = get_data(train_path)
# test_path = os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Test_Full')
# X_test, y_test = get_data(test_path)
# pickle.dump(X_data, open('data/X_data.pkl', 'wb'))
# pickle.dump(y_data, open('data/y_data.pkl', 'wb'))

# pickle.dump(X_test, open('data/X_test.pkl', 'wb'))
# pickle.dump(y_test, open('data/y_test.pkl', 'wb'))

100%|██████████████████████████████████████████████████████████████████████████████| 4560/4560 [01:22<00:00, 55.24it/s]


# Feature Engineering

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
X_data = pickle.load(open('data/X_data.pkl', 'rb'))
y_data = pickle.load(open('data/y_data.pkl', 'rb'))

X_test = pickle.load(open('data/X_test.pkl', 'rb'))
y_test = pickle.load(open('data/y_test.pkl', 'rb'))

## Bag - of - Words

- `\w`: Returns a match where the string contains any word characters (characters from a to Z, digits from 0-9, and the underscore _ character)
- The `r` means that the string is to be treated as a raw string, which means all escape codes will be ignored.
- `r'\w{1,}'`: từ chứa một ký tự chữ được lặp lại 1 lần hoặc hơn

In [9]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)

## TF-IDF Vectors
Trong các văn bản tương tự về chủ đề, lĩnh vực, các từ vựng thông dụng (generic term) sẽ xuất hiện nhiều, có thể lấn át đi các từ quan trọng, nếu chỉ đơn giản represent bằng word count.
### Ý tưởng:
- Các từ nào xuất hiện càng nhiều trong $n$ văn bản, khả năng nó là generic term càng cao, nên mình sẽ làm giảm bớt "mức độ ảnh hưởng" của nó đi bằng cho nó điểm thấp hơn, còn từ nào xuất hiện ở càng ít văn bản thì có khả năng nó là từ key để phân loại, ta cho điểm cao lên.
    - Công thức sử dụng công thức $IDF$
- Tuy nhiên, trong các văn bản một từ `t` có xuất hiện, số lượng từ vựng ở mỗi văn bản đó có thể chênh lệch nhau nhiều (một văn bản 5000 chữ so với một văn bản 10 chữ), thì nó cũng chưa chắc là từ key (mức độ quan trọng của cùng một từ khác nhau ở các văn bản khác nhau). Vì vậy ta cần sử dụng thêm công thức $TF$ để tính tần suất xuất hiện của từ ý trong một văn bản cụ thể $A$

$ TF(t)$ = (Number of times term t appears in a document) / (Total number of terms in the document)

$IDF(t)$ = log_e(Total number of documents / Number of documents with term t in it)

- TF-IDF Vectors can be generated at different levels of input tokens (words, characters, n-grams)

If a word appears in all the documents, we want it at the bottom of the range of 0–1. So, a logarithmic scale intuitively makes sense to be used here as log 1 is 0. However, there are some practical considerations such as avoiding the infamous divide by 0 error, 1 is added to the denominator.

Inverse Document frequency for the default settings in TF IDF vectorizer in sklearn is calculated as below (default settings have smooth_idf=True that adds “1” to the numerator and denominator as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions).

a. Word Level TF-IDF : Thực hiện tính toán dựa trên mỗi thành phần là một từ riêng lẻ

In [10]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)

b. N-gram Level TF-IDF : Kết hợp n thành phần (từ) liên tiếp nhau

In [11]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

c. Character Level TF-IDF : Dựa trên n-gram của ký tự.

In [12]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_data)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)

## Sử dụng thuật toán SVD (singular value decomposition) nhằm mục đích giảm chiều dữ liệu của ma trận

a. Word Level TF-IDF : Thực hiện tính toán dựa trên mỗi thành phần là một từ riêng lẻ

In [13]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

b. N-gram Level TF-IDF : Kết hợp n thành phần (từ) liên tiếp nhau

In [14]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

c. Character Level TF-IDF : Dựa trên n-gram của ký tự.

In [15]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)


# Label Encoder

In [16]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ # kết quả: array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
                 #                 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
                 #                 'Vi tinh'], dtype='<U16')


array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

# Model

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))


## Naive Bayes

Một giả thuyết rằng các chiều của dữ liệu $X= (x_1, x_2, x_3,\dots,x_n)$ là độc lập về mặt xác suất với nhau

In [28]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

# kết quả
# Train accuracy:  0.880031596616529
# Validation accuracy:  0.8690758293838863
# Test accuracy:  0.8650666031405714


Validation accuracy:  0.8524881516587678
Test accuracy:  0.8628034859944812


Với dữ liệu đã được SVD

In [29]:
train_model(naive_bayes.MultinomialNB(), X_data_tfidf_ngram_svd, y_data, X_test_tfidf_ngram_svd, y_test, is_neuralnet=False)

train_model(naive_bayes.MultinomialNB(), X_data_tfidf_ngram_char_svd, y_data, X_test_tfidf_ngram_char_svd, y_test, is_neuralnet=False)

ValueError: Negative values in data passed to MultinomialNB (input X)

### Dạng Bernoulli

In [30]:
train_model(naive_bayes.BernoulliNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.8485995457986374
# Validation accuracy:  0.8293838862559242
# Test accuracy:  0.8531554602664125


Validation accuracy:  0.8204976303317536
Test accuracy:  0.8517459750263038


In [31]:
train_model(naive_bayes.BernoulliNB(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.8087746437152354
# Validation accuracy:  0.8033175355450237
# Test accuracy:  0.8143449864014453


Validation accuracy:  0.8024289099526066
Test accuracy:  0.8149206916403628


## SVM Model

In [32]:
train_model(svm.SVC(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Validation accuracy:  0.9129146919431279
Test accuracy:  0.9186865979790761
